# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst in order to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Data Analyst",
    "task": "Continuously monitor and analyze market data for AAPL",
    "context": "The goal is to use statistical modeling and machine learning to identify trends and predict market movements for AAPL. This includes analyzing historical data, current market conditions, and any relevant news or events that may impact the stock."
}


> Entering new CrewAgentExecutor chain...
I need to gather as much data as possible to accurately monitor and analyze market data for A

Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
 News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science 2024 election Originals The 360 Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Shopping Finance My Portfolio My Portfolio News Latest News Stock Market Originals Premium News Economics Housing Earnings Tech Crypto Biden Economy EV Deep Dive Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Calendars Trending Tickers Futures World Indices US Treasury Bonds Currencies Crypto Top ETFs Top Mutual Funds Highest Open Interest Highest Implied Volatility Stock Comparison Advanced C

Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
 News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science 2024 election Originals The 360 Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Shopping Finance My Portfolio My Portfolio News Latest News Stock Market Originals Premium News Economics Housing Earnings Tech Crypto Biden Economy EV Deep Dive Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Calendars Trending Tickers Futures World Indices US Treasury Bonds Currencies Crypto Top ETFs Top Mutual Funds Highest Open Interest Highest Implied Volatility Stock Comparison Advanced C

I need to leverage the expertise of my team members to develop and refine trading strategies for AAPL that align with the user's risk tolerance and trading preferences.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Provide insights on AAPL stock and market trends", "context": "I need detailed analysis on AAPL stock performance, market trends, and any relevant news that could impact trading strategies."}

> Entering new CrewAgentExecutor chain...
I need to gather information on AAPL stock performance, market trends, and relevant news to provide detailed analysis to my coworker.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/AAPL"} 

Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
 News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science 2024 election Originals The 360 Life Health COVID-19 Fall a

Final Answer: Apple Inc. (AAPL) is currently trading at $189.87 with a slight increase of 0.02%. The company designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. As of May 17, 2024, Apple's market cap is $2.911 trillion, with a profit margin of 26.31% and a return on equity of 147.25%. The stock has shown a YTD return of 1.12% compared to the S&P 500's 11.18% return. Additionally, Apple offers various services such as Apple Music, Apple TV+, and Apple Pay, and operates platforms like the App Store. The company serves consumers, small and mid-sized businesses, and education, enterprise, and government markets.

> Finished chain.
 

Apple Inc. (AAPL) is currently trading at $189.87 with a slight increase of 0.02%. The company designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. As of May 17, 2024, Apple's market cap is $2.911 trillion, with a profit margin of 26.31%

Final Answer: Page Not Found - Unable to access information on Long-Term Investment Strategy from the provided website. I will need to search the internet for alternative sources to gather information on this strategy for AAPL.

> Finished chain.
 

Page Not Found - Unable to access information on Long-Term Investment Strategy from the provided website. I will need to search the internet for alternative sources to gather information on this strategy for AAPL.

Final Answer: I apologize for the inconvenience, but I was unable to gather detailed execution plans for AAPL trading strategies from the Trading Strategy Developer due to a technical issue. I recommend conducting further research or seeking input from other team members to determine the best execution methods for AAPL based on the approved trading strategies provided by the Data Analyst.

> Finished chain.
 [DEBUG]: [Crew Manager] Task output: I apologize for the inconvenience, but I was unable to gather detailed execution plans

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL Trading Strategies and Execution Plans:

After conducting a detailed analysis of the proposed trading strategies and execution plans for AAPL, the following potential risks have been identified:

1. Market Risk: The proposed strategies involve options trading, which is inherently risky due to market fluctuations. Changes in the price of AAPL stock can have a significant impact on the profitability of the strategies.

2. Volatility Risk: Volatility in the stock market can lead to unexpected price movements, affecting the performance of the trading strategies. Strategies involving options are particularly sensitive to volatility changes.

3. Liquidity Risk: The ability to execute trades at desired prices may be hindered by low liquidity in the options market for AAPL. This can result in slippage and increased transaction costs.

4. Counterparty Risk: Trading options involves counterparties, and the failure of a counterparty to fulfill their obligations can result in losses for the trading strategies.

5. Execution Risk: The lack of detailed execution plans for AAPL trading strategies introduces uncertainty in the implementation process, potentially leading to suboptimal outcomes.

To mitigate these risks, the following recommendations are suggested:

1. Diversification: Spread out the risk by using a combination of different trading strategies to reduce the impact of adverse market movements.

2. Risk Management: Implement risk management techniques such as stop-loss orders and position sizing to limit potential losses.

3. Monitoring: Regularly monitor the performance of the trading strategies and adjust them as needed based on market conditions.

4. Collaboration: Seek input from other team members, such as the Trading Strategy Developer and Data Analyst, to gather additional insights and refine the execution plans.

By addressing these risks and implementing the recommended mitigation strategies, the team can enhance the overall effectiveness and success of the AAPL trading strategies.